In [17]:
import re
import json
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
# Import things that are needed generically
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from pydantic import BaseModel, Field


In [19]:
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo-0613')

In [20]:
tools = []

'name'

In [21]:
def company_query_func(args):
    name = (args if isinstance(args, str) else args.get('name')).lower()
    members = {
        "james": "James is a member of esri company",
        "jim": "Jim is a member of google company",
        "jimmy": "Jimmy is a member of openai company",
        "jimothy": "Jimothy is a member facebook company",
    }
    return f"{members[name]}"


company_query_tool = Tool(**{
    "name": "CompanyQuery",
    "func": company_query_func,
    "description": "useful for when you want to know someone's company"
})

class CompanyQueryInfo(BaseModel):
    name: str = Field()

tools.append(
    Tool.from_function(
        func=company_query_func,
        name="CompanyQuery",
        description="useful for when you want to know someone's company",
        args_schema=CompanyQueryInfo,
    )
)

In [22]:
def company_info_func(args):
    company = (args if isinstance(args, str) else args.get('company')).lower()
    companys = {
        "esri": "esri is a geo related company",
        "google": "google is a search engine related company",
    }
    return f"{companys[company]}"

from pydantic import BaseModel, Field


class CompanyInfo(BaseModel):
    company: str = Field()

tools.append(
    Tool.from_function(
        func=company_info_func,
        name="GetCompanyInfo",
        description="useful for when you want to know a company's info",
        args_schema=CompanyInfo
    )
)

In [23]:
agent = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=False)

In [24]:
input = 'James belongs to which company? Introduce the company\'s information'
agent.run(input=input)

promot------------------------------
Respond to the human as helpfully and accurately as possible. You have access to the following tools:

CompanyQuery: useful for when you want to know someone's company, args: {{'name': {{'title': 'Name', 'type': 'string'}}}}
GetCompanyInfo: useful for when you want to know a company's info, args: {{'company': {{'title': 'Company', 'type': 'string'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or CompanyQuery, GetCompanyInfo

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final respons

'Esri is a geo-related company.'